##### 텍스트를 임베딩으로 변환
1. 토큰화(tokenization) 수행
    : 텍스트를 적절한 단위로 잘라 숫자형 id를 부여  
2. 토큰 임베딩 층
    : 토큰 id를 토큰 임베딩 layer를 통해 여러 숫자의 집합인 토큰 임베딩으로 변환  
3. 위치 인코딩 층
    : 토큰의 위치 정보를 담고 있는 위치 임베딩을 추가하여 최종적으로 모델에 입력할 임베딩을 만듬  

- 토큰화
    - 큰 단위: 텍스트의 의미 유지 good, 사전의 크기가 커짐, OOV(Out Of Vocabulary) 문제 발생
    - 작은 단위: OOV문제 없음, 텍스트 의미 거의 사라짐.  
    -> 장단점이 뚜렷함. 최근에는 데이터에 등장하는 빈도에 따라 토큰화 단위를 결정하는 subword 토큰화 방식 사용.  
        자주 나오는 단어 -> 단어 단위 유지  
        드물게 나오는 단어 -> 작게 나눔 -> 사전 크기가 커지지 않도록  

In [3]:
# 띄어쓰기 단위로 분리하는 토큰화 코드
input_text = "나는 최근 파리 여행을 다녀왔다"
input_text_list = input_text.split()
print("input_text_list: ", input_text_list)

# 토큰 -> 아이디 딕셔너리와 아이디 -> 토큰 딕셔너리 만들기
str2idx = {word:idx for idx, word in enumerate(input_text_list)}
idx2str = {idx:word for idx, word in enumerate(input_text_list)}
print("str2idx: ", str2idx)
print("idx2str: ", idx2str)

# 토큰을 토큰 아이디로 변환
input_ids = [str2idx[word] for word in input_text_list]
print("input_ids: ", input_ids)

input_text_list:  ['나는', '최근', '파리', '여행을', '다녀왔다']
str2idx:  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
idx2str:  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}
input_ids:  [0, 1, 2, 3, 4]


위의 토큰화에서 부여한 토큰 아이디는 그저 하나의 숫자일 뿐, 토큰의 의미를 담을 수 없음.  
의미를 담기 위해서는 최소2개 이상의 숫자 집합인 vector여야함.  
-> 데이터의 의미를 담아 숫자 집합으로 변환하는 것 = 임베딩(embedding)

In [4]:
# 토큰 임베딩 (토큰 아이디 -> 벡터)
import torch
import torch.nn as nn

embedding_dim = 16
# 사전 크기가 ?이고, 차원이 ?인 embed_layer 생성
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

# 입력 토큰을 임베딩으로 변환
# 즉, 토큰 1개를 16차원의 벡터로 변환
input_embeddings = embed_layer(torch.tensor(input_ids))     # (5, 16)
input_embeddings = input_embeddings.unsqueeze(0)        #(1, 5, 16)
input_embeddings.shape

torch.Size([1, 5, 16])

위의 코드에서 embed_layer는 그저 input_ids를 16차원의 임의의 숫자 집합으로 바꿔줄 뿐, 토큰의 의미가 담겨있는 벡터가 아님.  
-> 의미를 담기 위해서는 딥러닝 모델이 학습 데이터로 훈련되어야 함.  
-> 즉, 전체 모델이 학습되는 과정에서 임베딩 층도 학습되면서, 토큰의 의미를 잘 담은 임베딩을 생성하게 되는 것.

##### 위치 인코딩
- Transformer 모델은 모든 입력을 동시에 처리하기 때문에, 순서 정보가 사라진다.
- 텍스트에서 순서는 매우 중요한 정보이다.
- 이를 추가해주는 역할이 위치 인코딩이다.
- 위치 인코딩도 위치에 따른 인코딩 layer를 추가해 학습 데이터를 통해 학습하는 방식을 많이 활용하고 있다.
    - 입력 토큰의 위치에 따라 고정된 임베딩을 더해주는 것: 절대적 위치 인코딩(absolute position encoding)
        - 간단하게 구현할 수 있지만, 토큰과 토큰 사이의 상대적인 위치 정보는 활용하지 못하고, 긴 텍스트를 추론하는 경우에는 성능이 떨어진다.
    -> 최근에는 상대적 위치 인코딩(relative positinon encoding) 방식도 많이 활용된다.

In [5]:
# 절대적 위치 인코딩

embedding_dim = 16
# 최대 토큰 수
max_position = 12
embed_layer = nn.Embedding(len(str2idx), embedding_dim)
position_embed_layer = nn.Embedding(max_position, embedding_dim)

# 위치 id
position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
# 위치 인코딩 생성
position_encodings = position_embed_layer(position_ids)
# 토큰 임베딩 생성
token_embeddings = embed_layer(torch.tensor(input_ids))     # (5, 16)
token_embeddings = token_embeddings.unsqueeze(0)        # (1, 5, 16)
# 최종 입력 임베딩 (토큰 + 위치)
input_embeddings = token_embeddings + position_encodings
input_embeddings.shape

torch.Size([1, 5, 16])